# Step 1<br>
### Input: path directories to each folder containing data<br>
### Output: merged dataframes for analysis in Step 2<br>


In [19]:
folders = [
    '/Users/kateschnee/Library/CloudStorage/OneDrive-SharedLibraries-VUMC/Ellis, Sam - Collagen_Alignment_Project/Refined analysis pipeline/TR11_206',
    '/Users/kateschnee/Library/CloudStorage/OneDrive-SharedLibraries-VUMC/Ellis, Sam - Collagen_Alignment_Project/Refined analysis pipeline/TR11_16184_2',
    '/Users/kateschnee/Library/CloudStorage/OneDrive-SharedLibraries-VUMC/Ellis, Sam - Collagen_Alignment_Project/Refined analysis pipeline/TR11_21723',
    '/Users/kateschnee/Library/CloudStorage/OneDrive-SharedLibraries-VUMC/Ellis, Sam - Collagen_Alignment_Project/Refined analysis pipeline/TR16_23542',
    '/Users/kateschnee/Library/CloudStorage/OneDrive-SharedLibraries-VUMC/Ellis, Sam - Collagen_Alignment_Project/Refined analysis pipeline/TR11_18105'
]

In [20]:
import pandas as pd
import numpy as np
import sys
import csv
import os
import matplotlib.pyplot as plt
from scipy.stats import *
from tabulate import tabulate
import warnings
import pandoc
import time
import math
import pprint

In [21]:
start_time = time.time()

In [22]:
path_out = '/Users/kateschnee/Desktop/tmp/'

In [23]:
def progress_meter(i, n, e = 'blank'):
    if e != 'blank':
        print(f'{e}: {math.ceil(100 * i / n)}%', end = '\r')
    else:
        print(f'{math.ceil(100 * i / n)}%', end = '\r')

In [24]:
# input: directory path for a sample (ex. /Users/.../TR11_206)
# output: directory path for the same sample's ST data (ex. /Users/.../TR11_206/TR11_206_ST)
def get_folder_data_path(path_in):
    split_path = path_in.split("/")
    split_path.append(split_path[-1] + "_ST")
    return "/".join(split_path)

In [25]:
# input: directory path for a sample's ST data (ex. /Users/.../TR11_206/TR11_206_ST)
# reads through each of the files that ends in .tsv (stroma, epithelium, full, alignment)
# we only care about stroma, epithelium, and alignment
# loads the tsv data as a dataframe
# output: updates the overarching dictionary all_data to include the dataframe
# example: TR11_206: {stroma: stroma_dataframe}
#                    {epithelium: epithelium_dataframe}
#                    {alignment: alignment dataframe}
#          TR11_16184_2: 
# ...

def populate_first_dict(path_in):
        
    expt_name = path_in.split("/")[-2]
    dict_temp = {}
    for filename in os.listdir(folder):
    
        # create temporary variables to store data
        data_temp = []
        cols_temp = []
        
        
        # parse through the tsv files -- there should only be 4
        if filename.endswith(".tsv"):
            
            key = filename.split('.')[0]
                
            df = pd.read_csv(folder + '/' + filename, sep = '\t')

            # make sure index loads, otherwise assign it
            if df.index.tolist() == list(range(1, len(df.index) + 1)):
                df.set_index('barcodes', inplace = True)
            
            if "alignment" in key:
                dict_temp.update({"alignment": df})
            
            elif "stroma" in key:
                dict_temp.update({"stroma": df})
                                
            elif "epithelium" in key:                
                dict_temp.update({"epithelium": df})                

    return dict_temp

In [26]:
# all_data is a dictionary to store all information that the user inputs (see above functions)
all_data = {}

# iterate through each file directory that the user specifies, access its ST data, then load dataframe
for item in folders:
    
    # get the path to the subfolder containing csv files
    folder = get_folder_data_path(item)
    
    # get data into dictionary    
    all_data.update({item.split('/')[-1]: populate_first_dict(folder)})

In [27]:
# access each sample's epithelial and stromal data
# if a barcode is not in the alignment dataframe, we won't have x-y coordinates, so drop it
for expt, spot in all_data.items():
    for k, df in spot.items():
        if k == 'epithelium' or k == 'stroma':
            spot.update({k: df[df.index.isin((spot['alignment'].index.tolist()))]})

In [28]:
# input: stromal barcode (i), entire epithelial dataframe (epi_df), entire alignment dataframe (al_df)
# accesses the stromal barcode's x-y position then compares it to every epithelial x-y
# records lowest distance and retains that barcode
def get_barcode_pair(base_barcode, comparison_df, alignment_df):
    # arbitrary variables to get overwritten
    min_dist = 9999999999
    best_bc = ''
    
    # get base barcode's x-y data
    base_x = alignment_df.loc[base_barcode, 'pxl_col_in_fullres']
    base_y = alignment_df.loc[base_barcode, 'pxl_row_in_fullres']
    
    # iterate through every row in the comparison data
    for comparison_barcode, comparison_data in comparison_df.iterrows():
 
        # get comparison x-y
        comparison_x = alignment_df.loc[comparison_barcode, 'pxl_col_in_fullres']
        comparison_y = alignment_df.loc[comparison_barcode, 'pxl_row_in_fullres']
        
        # calculate distance
        dist = math.sqrt((comparison_x - base_x) ** 2 + (comparison_y - base_y) ** 2)
        
        # potentially overwrite data if applicable
        if dist < min_dist:
            min_dist = dist
            best_bc = comparison_barcode
    
    return min_dist, best_bc

In [29]:
# calls method to pair barcodes
# inputs: base_region: the region acting as a "base," either epithelium or stroma
#         comparison_region: the region that is being iterated through every spot; the other region than above
#         alignment_data: alignment values for the sample in question
def parse_barcodes(base_region, comparison_region, alignment_data, expt, location):
    i = 0
    data_temp = []
    
    for index, row_data in base_region.iterrows():
        i += 1
        
        # function to identify the epithelial partner
        distance, barcode_match = get_barcode_pair(index, comparison_region, alignment_data)
        
        # record the stromal barcode (index), epithelial match (ebc), and the distance (d) for use in a df
        data_temp.append([index, barcode_match, distance])
        
        # update progress for this sample
        progress_meter(i, base_region.shape[0], expt + ' ' + location)
    print()
    df = pd.DataFrame(data_temp)
    df.columns = [location + '_barcode', 'partner_barcode', 'distance']
    df.set_index(location + '_barcode', inplace = True)
    return df

In [30]:
# temporary: shorten dataframes so faster
# for expt, spot in all_data.items():
#     for k, df in spot.items():
#         if k == 'stroma':
#             spot.update({k: df.iloc[0:10]})

In [31]:
# for each sample, create a dataframe that acts as a legend
for expt, spot in all_data.items():
    stroma_pairings_df = parse_barcodes(spot['stroma'], spot['epithelium'], spot['alignment'], expt, 'stroma')
    epithelium_pairings_df = parse_barcodes(spot['epithelium'] ,spot['stroma'], spot['alignment'], expt, 'epithelium')
    all_pairings_df = pd.concat([stroma_pairings_df, epithelium_pairings_df])
    new_alignment = spot['alignment'].join(all_pairings_df, how = 'left')
    new_alignment.dropna(inplace = True)
    #print(f'Original alignment dataframe: {spot['alignment'].shape[0]}')
    #print(f'Original epithelium dataframe: {spot['epithelium'].shape[0]}')
    #print(f'Original stroma dataframe: {spot['stroma'].shape[0]}')
    #print(f'Final alignment dataframe: {new_alignment.shape[0]}')
    spot.update({'alignment': new_alignment})

    for k, df in spot.items():
        df.to_csv(path_out + expt + '_' + k + '.csv')
    


    

TR11_206 stroma: 100%
TR11_206 epithelium: 100%
TR11_16184_2 stroma: 13%

KeyboardInterrupt: 

In [ ]:
# # temporary: load the two data sets
# all_data = {}
# all_data.update({'TR11_206': {'epithelium': pd.read_csv(path_out + 'TR11_206_epithelium.csv', index_col = 0)}})
# all_data['TR11_206']['stroma'] = pd.read_csv(path_out + 'TR11_206_stroma.csv', index_col = 0)
# all_data['TR11_206']['alignment'] = pd.read_csv(path_out + 'TR11_206_alignment.csv', index_col = 0)
# all_data.update({'TR11_16184_2': {'epithelium': pd.read_csv(path_out + 'TR11_16184_2_epithelium.csv', index_col = 0)}})
# all_data['TR11_16184_2']['stroma'] = pd.read_csv(path_out + 'TR11_16184_2_stroma.csv', index_col = 0)
# all_data['TR11_16184_2']['alignment'] = pd.read_csv(path_out + 'TR11_16184_2_alignment.csv', index_col = 0)

In [ ]:
# for expt, spot in all_data.items():
#     for k, df in spot.items():

#         if k != 'alignment':
#             #df.columns.values[0] = 'barcode'
#             df.drop('barcodes', axis = 1, inplace = True)
#             spot.update({k: df})
            

In [79]:
def update_genes(dfe, dfs, gene_list):
    gene_list.extend(dfe.columns)
    gene_list.extend(dfs.columns)
    return list(set(gene_list))

In [80]:
# make a list of every gene to account for differences between samples
all_genes = []

# parse through each sample's ST data that was loaded previously
for expt, spot in all_data.items():

    epi_df, stroma_df = spot['epithelium'], spot['stroma']
    
    # add any new genes to the growing list to account for differences across samples
    all_genes = update_genes(epi_df, stroma_df, all_genes)

    # create alignment data and account for the non-gene data
    alignment_df = spot['alignment'].add_suffix('_non_gene')
    alignment_df.rename(columns = {'alignment_non_gene': 'alignment'}, inplace = True)

    # add genes that may be different with values of 0
    for g in all_genes:
        if g not in epi_df.columns:
            epi_df[g] = list(np.zeros(epi_df.shape[0]))
        if g not in stroma_df.columns:
            stroma_df[g] = list(np.zeros(stroma_df.shape[0]))

    spot.update({'alignment': alignment_df})

In [81]:
# input: either the epithelial or stromal dataframe (df1) and the alignment data (df2)
def merge_df(df1, df2):
    
    # merge the dataframes horizontally and only include overlapping indices (barcodes)
    new_df = pd.merge(df1, df2, left_index = True, right_index = True, how = 'inner')
    
    # remove the duplicate column "barcode" that might appear    
    if 'barcodes' in new_df.columns:
        new_df.drop('barcodes', axis = 1, inplace = True)
    
    
    return new_df

In [82]:
for expt, spot in all_data.items():
    # merge the epithelium and stromal data with the alignment data
    # now, we do not need to worry about the alignment dataframe

    epi_df, stroma_df, alignment_df = spot['epithelium'], spot['stroma'], spot['alignment']
    spot.update({'epithelium': merge_df(epi_df, alignment_df)})
    spot.update({'stroma': merge_df(stroma_df, alignment_df)})

In [83]:
for expt, spot in all_data.items():
    for k, df in spot.items():
        print(f'{expt} {k}: {df.shape}')

TR11_206 stroma: (1964, 18035)
TR11_206 epithelium: (852, 18035)
TR11_206 alignment: (2816, 10)
TR11_16184_2 stroma: (1467, 18049)
TR11_16184_2 epithelium: (1581, 18049)
TR11_16184_2 alignment: (3048, 10)
TR11_21723 stroma: (2605, 18064)
TR11_21723 alignment: (3750, 10)
TR11_21723 epithelium: (1145, 18064)
TR16_23542 stroma: (3333, 18064)
TR16_23542 alignment: (4564, 10)
TR16_23542 epithelium: (1231, 18064)
TR11_18105 stroma: (2845, 18065)
TR11_18105 alignment: (3276, 10)
TR11_18105 epithelium: (431, 18065)


In [84]:
def merge_dataframe(expt, df_in, alignment_df, existing_df):
    temp_df = df_in.reset_index()
    temp_df.rename(columns = {'index': 'barcode'}, inplace = True)
    temp_df.insert(0, 'experiment', [expt] * temp_df.shape[0])
    if df.shape[0] == 0:
        return temp_df
    else:
        return pd.concat([existing_df, temp_df])

In [85]:
# now, we want to combine the epithelial, stromal, and barcode-pairing dataframes across samples
dfe = pd.DataFrame()
dfs = pd.DataFrame()


# iterate through each sample's data
for expt, spot in all_data.items():

    dfe = merge_dataframe(expt, spot['epithelium'], spot['alignment'], dfe)
    dfs = merge_dataframe(expt, spot['stroma'], spot['alignment'], dfs)
        
dfe.set_index(['experiment', 'barcode'], inplace=True)
dfs.set_index(['experiment', 'barcode'], inplace=True)

In [87]:
dfe.to_csv(path_out + 'dfe.csv', index = True)
dfs.to_csv(path_out + 'dfs.csv', index = True)

In [88]:
end_time = time.time()
print(f'Total runtime: {math.floor((end_time - start_time)/60)} minutes and {math.floor((end_time - start_time)%60)} seconds.')

Total runtime: 435 minutes and 16 seconds.


In [91]:
for c in dfe.columns:
    if 'barcode' in c:
        print(c)

partner_barcode_non_gene


In [99]:
dfs_aligned = dfs.loc[pd.MultiIndex.from_arrays([
    dfe.index.get_level_values("experiment"),
    dfe["partner_barcode_non_gene"]
])].set_index(dfe.index)

cols_to_avoid = []
for c in dfe.columns:
    if c.endswith('_non_gene'):
        cols_to_avoid.append(c)
    elif 'barcode' in c:
        cols_to_avoid.append(c)
cols_to_avoid.append('experiment')

results = {}
for gene in dfe.columns:
    if gene not in cols_to_avoid:
        try:
            rho, pval = spearmanr(dfe[gene], dfs_aligned['alignment'])
            results[(gene, 'alignment')] = (rho, pval)
        except Exception as e:
            results[(gene, 'alignment')] = ('NA', 'NA')

# Tidy results table
results_df = (
    pd.DataFrame(results, index=["rho", "pval"])
    .T
    .reset_index()
    .rename(columns={"level_0": "dfe_gene", "level_1": "dfs_gene"})
)


results_df.to_csv(path_out + 'new_stroma_comparisons.csv')

/var/folders/m2/fxhwzr9d629bg5_5v13p70_00000gn/T/ipykernel_74342/2895656434.py:18: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho, pval = spearmanr(dfe[gene], dfs_aligned['alignment'])


         dfe_gene   dfs_gene       rho      pval
0          SAMD11  alignment  0.012491  0.365996
1           NOC2L  alignment -0.013606  0.324773
2          KLHL17  alignment -0.011974  0.386174
3         PLEKHN1  alignment -0.006535  0.636260
4           PERM1  alignment  0.027966  0.042941
...           ...        ...       ...       ...
18051      NLGN4Y  alignment       NaN       NaN
18052  AC007244.1  alignment       NaN       NaN
18053       KDM5D  alignment       NaN       NaN
18054      EIF1AY  alignment       NaN       NaN
18055      TCEAL5  alignment       NaN       NaN

[18056 rows x 4 columns]


In [ ]:
end_time = time.time()
print(f'Total runtime: {math.floor((end_time - start_time)/60)} minutes and {math.floor((end_time - start_time)%60)} seconds.')